In [1]:
from datetime import datetime
import pandas as pd
def load_accepted_loans(file_path: str) -> pd.DataFrame:
    df = pd.read_csv(file_path)
    print(f"{len(df):,} rows loaded from {file_path}")

    df['fico_score'] = (df['fico_range_low'] + df['fico_range_high']) / 2
    df = df.dropna(subset=['fico_score', 'earliest_cr_line', 'issue_d'])

    df['issue_d'] = pd.to_datetime(df['issue_d'], format='%b-%Y')
    df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], format='%b-%Y', errors='coerce')
    df['credit_history_years'] = ((df['issue_d'] - df['earliest_cr_line']).dt.days / 365).fillna(0)
    df = df[df['credit_history_years'] > 0]

    # ----- 📈 FICO Feature Engineering -----

    # 1. Credit Utilization (% of revolving credit used)
    df['revol_util'] = pd.to_numeric(df['revol_util'].astype(str).str.replace('%', '', regex=False), errors='coerce')
    df['credit_utilization'] = df['revol_util'] / 100


    # 2. Payment History
    df['total_delinquency'] = (
        df['delinq_2yrs'].fillna(0) +
        df['pub_rec'].fillna(0) +
        df['collections_12_mths_ex_med'].fillna(0)
    )

    # 3. New Credit
    df['recent_inquiries'] = df['inq_last_6mths'].fillna(0)

    # 4. Credit Mix
    df['credit_mix'] = (
        df['mort_acc'].fillna(0) +
        df['open_acc'].fillna(0) +
        df['num_actv_bc_tl'].fillna(0)
    )

    # 5. Total debt exposure
    df['total_balance'] = df['revol_bal'].fillna(0) + df['loan_amnt'].fillna(0)

    # Keep only relevant engineered features + target
    features_to_keep = [
        'fico_score', 'credit_utilization', 'credit_history_years','int_rate',
        'total_delinquency', 'recent_inquiries', 'credit_mix', 'total_balance'
    ]
    df = df[features_to_keep].dropna()

    print(f"After feature engineering, rows retained: {len(df):,}")
    return df


In [2]:
file_path = "../data/accepted_2007_to_2018Q4.csv"
accepted_df = load_accepted_loans(file_path)
accepted_df.to_csv("../data/processed/accepted_cleaned.csv", index=False)

C:\Users\Gpentela\AppData\Local\Temp\ipykernel_26144\102911400.py:4: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


2,260,701 rows loaded from ../data/accepted_2007_to_2018Q4.csv
After feature engineering, rows retained: 2,258,866


In [3]:
accepted_df

,fico_score,credit_utilization,credit_history_years,int_rate,total_delinquency,recent_inquiries,credit_mix,total_balance
0,677.0,0.297,12.342466,13.99,0.0,1.0,10.0,6365.0
1,717.0,0.192,16.010959,11.99,1.0,4.0,31.0,46170.0
2,697.0,0.562,15.342466,10.78,0.0,0.0,13.0,27869.0
3,787.0,0.116,7.252055,14.85,0.0,0.0,18.0,42802.0
4,697.0,0.645,17.512329,22.45,1.0,3.0,22.0,32329.0
...,...,...,...,...,...,...,...,...
2260694,667.0,0.844,16.846575,12.79,0.0,0.0,5.0,73431.0
2260695,697.0,0.390,25.682192,10.49,1.0,1.0,33.0,45665.0
2260696,707.0,0.649,21.679452,10.49,7.0,1.0,10.0,48633.0
2260697,662.0,0.681,17.265753,14.49,1.0,0.0,17.0,41641.0
